# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_load = pd.read_csv("output_data/cities1.csv")
weather_data_load

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp (F),Wind Speed (mph)
0,Carnarvon,0,AU,1613013558,61,-24.8667,113.6333,80.60,17.27
1,Salalah,0,OM,1613013284,38,17.0151,54.0924,73.40,9.22
2,Vaini,75,TO,1613013531,83,-21.2000,-175.2000,82.40,13.80
3,Albany,100,US,1613013354,47,42.6001,-73.9662,23.00,3.20
4,Port-Gentil,96,GA,1613013216,80,-0.7193,8.7815,82.17,7.18
...,...,...,...,...,...,...,...,...,...
553,Kangaatsiaq,29,GL,1613013652,96,68.3065,-53.4641,13.51,9.42
554,Balkhash,0,KZ,1613013652,85,46.8440,74.9804,-0.40,8.95
555,Diego de Almagro,0,CL,1613013245,80,-26.3667,-70.0500,55.85,0.76
556,Punta Gorda,20,US,1613013496,88,26.9298,-82.0454,72.00,2.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)
loc_store = weather_data_load[["Lat","Lng"]]
humidity = weather_data_load["Humidity"]

In [4]:
#Add heatmap layer
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(loc_store, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the dataframe with max temp between 70 and 80, wind speed less than 10 mph, and 0 cloudiness 
narrowed_cities_df = weather_data_load.loc[(weather_data_load["Wind Speed (mph)"] < 10) & (weather_data_load["Cloudiness"] == 0) & \
(weather_data_load["Max Temp (F)"] > 70) & (weather_data_load["Max Temp (F)"] < 80)].dropna()

narrowed_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp (F),Wind Speed (mph)
1,Salalah,0,OM,1613013284,38,17.0151,54.0924,73.40,9.22
53,Gorom-Gorom,0,BF,1613013566,17,14.4439,-0.2361,72.28,8.14
57,General Roca,0,AR,1613013566,38,-39.0333,-67.5833,75.20,8.05
92,Sennar,0,SD,1613013572,23,13.5691,33.5672,70.32,8.48
108,Pisco,0,PE,1613013264,78,-13.7000,-76.2167,73.40,5.75
142,Saint-Philippe,0,RE,1613013008,69,-21.3585,55.7679,77.00,4.61
240,Sikasso,0,ML,1613013596,21,10.9167,-7.0000,71.60,3.60
246,Puerto Colombia,0,CO,1613013597,78,10.9878,-74.9547,78.80,6.91
249,Tamale,0,GH,1613013598,16,9.4008,-0.8393,75.92,3.15
300,Agnibilékrou,0,CI,1613013607,98,7.1344,-3.2047,71.31,2.28


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrowed_cities_df.loc[:,["City","Country","Lat","Lng"]]

In [8]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
1,Salalah,OM,17.0151,54.0924,
53,Gorom-Gorom,BF,14.4439,-0.2361,
57,General Roca,AR,-39.0333,-67.5833,
92,Sennar,SD,13.5691,33.5672,
108,Pisco,PE,-13.7000,-76.2167,
142,Saint-Philippe,RE,-21.3585,55.7679,
240,Sikasso,ML,10.9167,-7.0000,
246,Puerto Colombia,CO,10.9878,-74.9547,
249,Tamale,GH,9.4008,-0.8393,
300,Agnibilékrou,CI,7.1344,-3.2047,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [13]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name_var = row["City"]
    
    params["location"] = f"{lat},{lng}"

    print(f"Retrieving Results for Index {index}: {city_name_var}.")
    response = requests.get(base_url, params=params).json()
    
    results = response["results"]
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        print(f"Closest hotel in {city_name_var} is {results[0]['name']}.")

    except (KeyError, IndexError):
        print("Missing result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request in order to avoid an SSL Error
    time.sleep(1)

# Print end of search when completed
print("-------Search Complete-------")

Retrieving Results for Index 1: Salalah.
Closest hotel in Salalah is Star Emirates garden villas.
------------
Retrieving Results for Index 53: Gorom-Gorom.
Closest hotel in Gorom-Gorom is Mairie de Rambo.
------------
Retrieving Results for Index 57: General Roca.
Closest hotel in General Roca is Estepa Hotel.
------------
Retrieving Results for Index 92: Sennar.
Closest hotel in Sennar is فندق الرشيد.
------------
Retrieving Results for Index 108: Pisco.
Closest hotel in Pisco is HOSTAL MIRAMAR.
------------
Retrieving Results for Index 142: Saint-Philippe.
Closest hotel in Saint-Philippe is Les Embruns Du Baril.
------------
Retrieving Results for Index 240: Sikasso.
Missing result... skipping.
------------
Retrieving Results for Index 246: Puerto Colombia.
Closest hotel in Puerto Colombia is Hotel Pradomar.
------------
Retrieving Results for Index 249: Tamale.
Closest hotel in Tamale is Jeyads Lodge.
------------
Retrieving Results for Index 300: Agnibilékrou.
Closest hotel in Agn

In [14]:
#Display the DF with Hotel Name
hotel_df

,City,Country,Lat,Lng,Hotel Name
1,Salalah,OM,17.0151,54.0924,Star Emirates garden villas
53,Gorom-Gorom,BF,14.4439,-0.2361,Mairie de Rambo
57,General Roca,AR,-39.0333,-67.5833,Estepa Hotel
92,Sennar,SD,13.5691,33.5672,فندق الرشيد
108,Pisco,PE,-13.7000,-76.2167,HOSTAL MIRAMAR
142,Saint-Philippe,RE,-21.3585,55.7679,Les Embruns Du Baril
240,Sikasso,ML,10.9167,-7.0000,
246,Puerto Colombia,CO,10.9878,-74.9547,Hotel Pradomar
249,Tamale,GH,9.4008,-0.8393,Jeyads Lodge
300,Agnibilékrou,CI,7.1344,-3.2047,HOTEL KOUABIA


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))